In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# reading in initial data - gives player data for each GW of the season from 16/17 to 22/23
data = pd.read_csv("merged_seasons.csv")

/tmp/ipykernel_12727/3350283255.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("merged_seasons.csv")


In [70]:
data

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,...,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,...,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,...,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,...,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96164,2022-23,Oliver Skipp,MID,Spurs,0,0,16,0,0.0,441,...,1.0,0.0,2,100,742,642,43,False,0,38
96165,2022-23,Ryan Sessegnon,DEF,Spurs,0,0,0,0,0.0,436,...,1.0,0.0,0,-166,24,190,44,False,0,38
96166,2022-23,Ashley Young,DEF,Aston Villa,0,0,0,0,0.0,538,...,2.0,0.0,0,-1146,1522,2668,43,True,0,38
96167,2022-23,Jeremy Sarmiento Morante,MID,Brighton,0,0,0,0,0.0,119,...,2.0,0.0,0,-17,22,39,45,False,0,38


In [71]:
np.unique(data["season_x"])

array(['2016-17', '2017-18', '2020-21', '2021-22', '2022-23'],
      dtype=object)

In [72]:
data.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [73]:
# reading in the list of player IDs to convert IDs to names
player_IDs = pd.read_csv("player_IDs.csv")
IDs = player_IDs["id"]

In [74]:
player_IDs

,first_name,second_name,id
0,Folarin,Balogun,1
1,Cédric,Alves Soares,2
2,Mohamed,Elneny,3
3,Fábio,Ferreira Vieira,4
4,Gabriel,dos Santos Magalhães,5
...,...,...,...
753,Nathan,Fraser,704
754,Jean-Ricner,Bellegarde,715
755,Matthew,Whittingham,749
756,Tawanda,Chirewa,750


In [75]:
player_IDs[player_IDs["id"] == 355]

,first_name,second_name,id
468,Erling,Haaland,355


In [76]:
IDs

0        1
1        2
2        3
3        4
4        5
      ... 
753    704
754    715
755    749
756    750
757    751
Name: id, Length: 758, dtype: int64

In [2]:
driver_path = "/home/jasmine/Desktop/drivers/chromedriver-linux64/chromedriver"
driver = webdriver.Chrome(executable_path=driver_path)

In [3]:
player_link = "https://www.premierleague.com/players"
# direct the driver to the page
driver.get(player_link)

In [4]:
# click the accept all cookies button - one-time
cookie_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))

cookie_button.click()

In [5]:
search_input = driver.find_element(By.ID, "search-input")
search_input.clear()
# verify we got the correct element
print(search_input.get_attribute("outerHTML"))

<input id="search-input" type="text" value="" placeholder="Search for a Player" class="searchTextContainer searchInput">


In [6]:
search_input.send_keys("Gabriel Jesus")
search_input.send_keys(Keys.RETURN)

In [7]:
search_results = WebDriverWait(driver, 3).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "player__name"))
)
first_result = search_results[0]
first_result.click()

In [8]:
# get the details
overview_info = driver.find_elements(By.CLASS_NAME, "player-overview__col")
club = overview_info[3].text.split("\n")[1]
position = overview_info[4].text.split("\n")[1]
club, position

('Arsenal', 'Forward')

In [12]:
driver.back()
search_input.clear()

In [9]:
# use BeautifulSoup
page_html = driver.page_source
soup = BeautifulSoup(page_html, features = "html.parser")

In [10]:
# getting rid of the shortened names to make the df easier to use
short_names = soup.find_all("span", class_="player-club-history__team-name--short")
for short_name in short_names:
    short_name.decompose()

In [11]:
# finding the table containing club history
table = soup.find("table")

In [23]:
clubs_df = pd.read_html(str(table))[0]
clubs_df

,Season,Club,Apps (Subs),Goals,More
0,2023/2024,Arsenal,11 (3),2,NaN
1,NaN,NaN,Other Competitions,Other Competitions,Other Competitions
2,NaN,EFL Cup,1 (0),0,NaN
3,NaN,English Community Shield,0 (0),0,NaN
4,NaN,FA Cup,0 (0),0,NaN
5,NaN,UEFA Champions League,5 (1),4,NaN
6,2022/2023,Arsenal,26 (2),11,NaN
7,NaN,NaN,Other Competitions,Other Competitions,Other Competitions
8,NaN,EFL Cup,1 (0),0,NaN
9,NaN,FA Cup,0 (0),0,NaN


In [24]:
clubs_df = clubs_df.dropna(subset=["Season"])
clubs_df

,Season,Club,Apps (Subs),Goals,More
0,2023/2024,Arsenal,11 (3),2,NaN
6,2022/2023,Arsenal,26 (2),11,NaN
11,2022/2023,Manchester City,0 (0),0,NaN
15,2021/2022,Manchester City,28 (7),8,NaN
21,2020/2021,Manchester City,29 (7),9,NaN
26,2019/2020,Manchester City,34 (13),14,NaN
32,2018/2019,Manchester City,29 (21),7,NaN
38,2017/2018,Manchester City,29 (10),13,NaN
44,2016/2017,Manchester City,10 (2),7,NaN


In [25]:
# some players who transfer to another PL club have both clubs under the same season; we only want one
clubs_df = clubs_df.drop_duplicates(subset=["Season"])
clubs_df

,Season,Club,Apps (Subs),Goals,More
0,2023/2024,Arsenal,11 (3),2,NaN
6,2022/2023,Arsenal,26 (2),11,NaN
15,2021/2022,Manchester City,28 (7),8,NaN
21,2020/2021,Manchester City,29 (7),9,NaN
26,2019/2020,Manchester City,34 (13),14,NaN
32,2018/2019,Manchester City,29 (21),7,NaN
38,2017/2018,Manchester City,29 (10),13,NaN
44,2016/2017,Manchester City,10 (2),7,NaN


In [42]:
# format: https://fantasy.premierleague.com/api/element-summary/{player_id}/
player_api_link = "https://fantasy.premierleague.com/api/element-summary/"

In [11]:
player_row = player_IDs[player_IDs["id"] == 1]
player_name = player_row["first_name"] + " " + player_row["second_name"]

response = requests.get(f"{player_api_link}{1}/")
api_data = response.json()
fixture_stats = api_data.get("history", [])
colnames = list(fixture_stats[1].keys())
colnames

['element',
 'fixture',
 'opponent_team',
 'total_points',
 'was_home',
 'kickoff_time',
 'team_h_score',
 'team_a_score',
 'round',
 'minutes',
 'goals_scored',
 'assists',
 'clean_sheets',
 'goals_conceded',
 'own_goals',
 'penalties_saved',
 'penalties_missed',
 'yellow_cards',
 'red_cards',
 'saves',
 'bonus',
 'bps',
 'influence',
 'creativity',
 'threat',
 'ict_index',
 'starts',
 'expected_goals',
 'expected_assists',
 'expected_goal_involvements',
 'expected_goals_conceded',
 'value',
 'transfers_balance',
 'selected',
 'transfers_in',
 'transfers_out']

In [83]:
to_add_df = pd.DataFrame(columns=["name"] + colnames)
for player_id in IDs: 
    player_row = player_IDs[player_IDs["id"] == player_id]
    player_name = f"{player_row['first_name'].iloc[0]} {player_row['second_name'].iloc[0]}"
    response = requests.get(f"{player_api_link}{player_id}/")
    api_data = response.json()
    fixture_stats = api_data.get("history", [])
    add_df = pd.DataFrame(fixture_stats)
    nrows = add_df.shape[0]
    names = [player_name] * nrows
    add_df.insert(0, column="name", value=names)
    to_add_df = pd.concat([to_add_df, add_df], ignore_index=True)

In [87]:
to_add_df

,name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,Folarin Balogun,1,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0,0.00,0.00,0.00,0.00,45,0,59090,0,0
1,Folarin Balogun,1,12,8,0,False,2023-08-21T19:00:00Z,0,1,2,...,0,0.00,0.00,0.00,0.00,45,-4744,63768,4959,9703
2,Folarin Balogun,1,21,10,0,True,2023-08-26T14:00:00Z,2,2,3,...,0,0.00,0.00,0.00,0.00,45,-7682,58109,2507,10189
3,Folarin Balogun,1,31,14,0,True,2023-09-03T15:30:00Z,3,1,4,...,0,0.00,0.00,0.00,0.00,44,-9696,49814,2558,12254
4,Folarin Balogun,1,43,9,0,False,2023-09-17T15:30:00Z,0,1,5,...,0,0.00,0.00,0.00,0.00,44,-12786,37048,0,12786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13409,Tawanda Chirewa,750,149,6,0,True,2023-12-05T19:30:00Z,1,0,15,...,0,0.00,0.00,0.00,0.00,45,92,138,109,17
13410,Tawanda Chirewa,750,160,16,0,True,2023-12-09T15:00:00Z,1,1,16,...,0,0.00,0.00,0.00,0.00,45,44,220,94,50
13411,Justin Hubner,751,131,1,0,False,2023-12-02T15:00:00Z,2,1,14,...,0,0.00,0.00,0.00,0.00,40,0,0,0,0
13412,Justin Hubner,751,149,6,0,True,2023-12-05T19:30:00Z,1,0,15,...,0,0.00,0.00,0.00,0.00,40,344,555,375,31


In [92]:
shared_cols = list(to_add_df.columns.intersection(data.columns))
shared_cols

['name',
 'element',
 'fixture',
 'opponent_team',
 'total_points',
 'was_home',
 'kickoff_time',
 'team_h_score',
 'team_a_score',
 'round',
 'minutes',
 'goals_scored',
 'assists',
 'clean_sheets',
 'goals_conceded',
 'own_goals',
 'penalties_saved',
 'penalties_missed',
 'yellow_cards',
 'red_cards',
 'saves',
 'bonus',
 'bps',
 'influence',
 'creativity',
 'threat',
 'ict_index',
 'value',
 'transfers_balance',
 'selected',
 'transfers_in',
 'transfers_out']

In [98]:
to_add_df = to_add_df.drop(columns=to_add_df.columns.difference(shared_cols))
data = data.drop(columns=data.columns.difference(shared_cols))

In [94]:
to_add_df

,name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,Folarin Balogun,1,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0,0.0,0.0,0.0,0.0,45,0,59090,0,0
1,Folarin Balogun,1,12,8,0,False,2023-08-21T19:00:00Z,0,1,2,...,0,0.0,0.0,0.0,0.0,45,-4744,63768,4959,9703
2,Folarin Balogun,1,21,10,0,True,2023-08-26T14:00:00Z,2,2,3,...,0,0.0,0.0,0.0,0.0,45,-7682,58109,2507,10189
3,Folarin Balogun,1,31,14,0,True,2023-09-03T15:30:00Z,3,1,4,...,0,0.0,0.0,0.0,0.0,44,-9696,49814,2558,12254
4,Folarin Balogun,1,43,9,0,False,2023-09-17T15:30:00Z,0,1,5,...,0,0.0,0.0,0.0,0.0,44,-12786,37048,0,12786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13409,Tawanda Chirewa,750,149,6,0,True,2023-12-05T19:30:00Z,1,0,15,...,0,0.0,0.0,0.0,0.0,45,92,138,109,17
13410,Tawanda Chirewa,750,160,16,0,True,2023-12-09T15:00:00Z,1,1,16,...,0,0.0,0.0,0.0,0.0,45,44,220,94,50
13411,Justin Hubner,751,131,1,0,False,2023-12-02T15:00:00Z,2,1,14,...,0,0.0,0.0,0.0,0.0,40,0,0,0,0
13412,Justin Hubner,751,149,6,0,True,2023-12-05T19:30:00Z,1,0,15,...,0,0.0,0.0,0.0,0.0,40,344,555,375,31


In [99]:
data

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards
0,Aaron Cresswell,0,0,0,0,0.0,454,10,0,0,...,1.0,2.0,0.0,0,0,0,0,55,False,0
1,Aaron Lennon,0,0,6,0,0.3,142,3,0,0,...,1.0,1.0,0.0,1,0,0,0,60,True,0
2,Aaron Ramsey,0,0,5,0,4.9,16,8,3,0,...,4.0,3.0,23.0,2,0,0,0,80,True,0
3,Abdoulaye Doucouré,0,0,0,0,0.0,482,7,0,0,...,1.0,1.0,0.0,0,0,0,0,50,False,0
4,Adam Forshaw,0,0,3,0,1.3,286,6,1,0,...,1.0,1.0,0.0,1,0,0,0,45,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96164,Oliver Skipp,0,0,16,0,0.0,441,377,1,0,...,4.0,1.0,0.0,2,100,742,642,43,False,0
96165,Ryan Sessegnon,0,0,0,0,0.0,436,377,0,0,...,4.0,1.0,0.0,0,-166,24,190,44,False,0
96166,Ashley Young,0,0,0,0,0.0,538,372,0,0,...,1.0,2.0,0.0,0,-1146,1522,2668,43,True,0
96167,Jeremy Sarmiento Morante,0,0,0,0,0.0,119,372,0,0,...,1.0,2.0,0.0,0,-17,22,39,45,False,0


In [100]:
to_add_df.dtypes, data.dtypes

(name                 object
 element              object
 fixture              object
 opponent_team        object
 total_points         object
 was_home             object
 kickoff_time         object
 team_h_score         object
 team_a_score         object
 round                object
 minutes              object
 goals_scored         object
 assists              object
 clean_sheets         object
 goals_conceded       object
 own_goals            object
 penalties_saved      object
 penalties_missed     object
 yellow_cards         object
 red_cards            object
 saves                object
 bonus                object
 bps                  object
 influence            object
 creativity           object
 threat               object
 ict_index            object
 value                object
 transfers_balance    object
 selected             object
 transfers_in         object
 transfers_out        object
 dtype: object,
 name                  object
 assists                in

In [102]:
are_numeric = list(data.select_dtypes(include=['number']).columns)
are_numeric

['assists',
 'bonus',
 'bps',
 'clean_sheets',
 'creativity',
 'element',
 'fixture',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'minutes',
 'opponent_team',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'round',
 'saves',
 'selected',
 'team_a_score',
 'team_h_score',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'yellow_cards']

In [105]:
# convert columns from object dtype to numeric/bool as in data df
to_add_df[are_numeric] = to_add_df[are_numeric].apply(pd.to_numeric)
to_add_df["was_home"] = to_add_df["was_home"].astype(bool)
to_add_df.dtypes

name                  object
element                int64
fixture                int64
opponent_team          int64
total_points           int64
was_home                bool
kickoff_time          object
team_h_score           int64
team_a_score           int64
round                  int64
minutes                int64
goals_scored           int64
assists                int64
clean_sheets           int64
goals_conceded         int64
own_goals              int64
penalties_saved        int64
penalties_missed       int64
yellow_cards           int64
red_cards              int64
saves                  int64
bonus                  int64
bps                    int64
influence            float64
creativity           float64
threat               float64
ict_index            float64
value                  int64
transfers_balance      int64
selected               int64
transfers_in           int64
transfers_out          int64
dtype: object

In [106]:
full_data = pd.concat([data, to_add_df], ignore_index=True)

In [107]:
full_data

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards
0,Aaron Cresswell,0,0,0,0,0.0,454,10,0,0,...,1.0,2.0,0.0,0,0,0,0,55,False,0
1,Aaron Lennon,0,0,6,0,0.3,142,3,0,0,...,1.0,1.0,0.0,1,0,0,0,60,True,0
2,Aaron Ramsey,0,0,5,0,4.9,16,8,3,0,...,4.0,3.0,23.0,2,0,0,0,80,True,0
3,Abdoulaye Doucouré,0,0,0,0,0.0,482,7,0,0,...,1.0,1.0,0.0,0,0,0,0,50,False,0
4,Adam Forshaw,0,0,3,0,1.3,286,6,1,0,...,1.0,1.0,0.0,1,0,0,0,45,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109578,Tawanda Chirewa,0,0,0,0,0.0,750,149,0,0,...,0.0,1.0,0.0,0,92,109,17,45,True,0
109579,Tawanda Chirewa,0,0,0,0,0.0,750,160,0,0,...,1.0,1.0,0.0,0,44,94,50,45,True,0
109580,Justin Hubner,0,0,0,0,0.0,751,131,0,0,...,1.0,2.0,0.0,0,0,0,0,40,False,0
109581,Justin Hubner,0,0,0,0,0.0,751,149,0,0,...,0.0,1.0,0.0,0,344,375,31,40,True,0


In [17]:
# full_data.to_csv("full_data.csv")
full_data = pd.read_csv("full_data.csv")
full_data

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
0,Aaron Cresswell,0,0,0,0,0.0,454,10,0,0,...,2.0,0.0,0,0,0,0,55,False,0,2016/2017
1,Aaron Lennon,0,0,6,0,0.3,142,3,0,0,...,1.0,0.0,1,0,0,0,60,True,0,2016/2017
2,Aaron Ramsey,0,0,5,0,4.9,16,8,3,0,...,3.0,23.0,2,0,0,0,80,True,0,2016/2017
3,Abdoulaye Doucouré,0,0,0,0,0.0,482,7,0,0,...,1.0,0.0,0,0,0,0,50,False,0,2016/2017
4,Adam Forshaw,0,0,3,0,1.3,286,6,1,0,...,1.0,0.0,1,0,0,0,45,True,1,2016/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109578,Tawanda Chirewa,0,0,0,0,0.0,750,149,0,0,...,1.0,0.0,0,92,109,17,45,True,0,2023/2024
109579,Tawanda Chirewa,0,0,0,0,0.0,750,160,0,0,...,1.0,0.0,0,44,94,50,45,True,0,2023/2024
109580,Justin Hubner,0,0,0,0,0.0,751,131,0,0,...,2.0,0.0,0,0,0,0,40,False,0,2023/2024
109581,Justin Hubner,0,0,0,0,0.0,751,149,0,0,...,1.0,0.0,0,344,375,31,40,True,0,2023/2024


In [79]:
full_data["name"].value_counts()

name
Ben Davies           299
Granit Xhaka         222
Tyrone Mings         222
Darren Randolph      222
Calum Chambers       222
                    ... 
Charlie Savage         1
Nile John              1
Sylvester Jasper       1
Sydie Peck             1
Samuel Amo-Ameyaw      1
Name: count, Length: 1532, dtype: int64

In [80]:
full_data["kickoff_time"] = pd.to_datetime(full_data["kickoff_time"])

In [81]:
full_data.dtypes

name                              object
assists                            int64
bonus                              int64
bps                                int64
clean_sheets                       int64
creativity                       float64
element                            int64
fixture                            int64
goals_conceded                     int64
goals_scored                       int64
ict_index                        float64
influence                        float64
kickoff_time         datetime64[ns, UTC]
minutes                            int64
opponent_team                      int64
own_goals                          int64
penalties_missed                   int64
penalties_saved                    int64
red_cards                          int64
round                              int64
saves                              int64
selected                           int64
team_a_score                     float64
team_h_score                     float64
threat          

In [18]:
unique_players = np.unique(full_data["name"])
unique_players

array(['Aaron Connolly', 'Aaron Cresswell', 'Aaron Hickey', ...,
       'Ângelo Gabriel Borges Damaceno', 'Çaglar Söyüncü', 'Ørjan Nyland'],
      dtype=object)

In [101]:
full_data["kickoff_time"][106121]

Timestamp('2023-08-12 16:30:00+0000', tz='UTC')

In [92]:
# function that assigns the season to each row
def assign_season(ko_time):

    dates = ["2017-06-01", "2018-06-01", "2019-06-01", "2020-06-01", "2021-06-01", "2022-06-01", "2023-06-01", "2024-06-01"]

    for date in dates: 

        if ko_time < pd.to_datetime(date).tz_localize("UTC"): 

            year1 = int(date.split("-")[0])
            year0 = year1 - 1

            season = f"{year0}/{year1}"
            
            return season

In [103]:
full_data['season'] = full_data["kickoff_time"].apply(assign_season)

In [105]:
full_data["season"].value_counts()

season
2022/2023    26505
2021/2022    25447
2020/2021    24365
2023/2024    13414
2017/2018    11285
2016/2017     8567
Name: count, dtype: int64

In [106]:
full_data.to_csv("full_data.csv", index=False)

In [257]:
full_data = pd.read_csv("full_data.csv")
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
0,Aaron Cresswell,West Ham United,Defender,0,0,0,0,0.0,454,10,...,2.0,0.0,0,0,0,0,55,False,0,2016/2017
1,Aaron Lennon,Everton,Midfielder,0,0,6,0,0.3,142,3,...,1.0,0.0,1,0,0,0,60,True,0,2016/2017
2,Abdoulaye Doucouré,Watford,Midfielder,0,0,0,0,0.0,482,7,...,1.0,0.0,0,0,0,0,50,False,0,2016/2017
3,Adam Forshaw,Middlesbrough,Midfielder,0,0,3,0,1.3,286,6,...,1.0,0.0,1,0,0,0,45,True,1,2016/2017
4,Adam Lallana,Liverpool,Midfielder,1,2,33,0,33.7,205,8,...,3.0,57.0,11,0,0,0,70,False,1,2016/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107854,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,10,0,14.4,715,120,...,2.0,3.0,2,389,465,76,49,True,0,2023/2024
107855,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,1,0,22,0,28.7,715,124,...,3.0,8.0,5,221,439,218,49,False,0,2023/2024
107856,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,6,0,1.7,715,131,...,2.0,2.0,2,952,1148,196,49,False,0,2023/2024
107857,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,3,0,0.8,715,149,...,1.0,2.0,1,934,1311,377,49,True,0,2023/2024


In [106]:
full_data.insert(1, "club", None)
full_data.insert(2, "position", None)
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
0,Aaron Cresswell,None,None,0,0,0,0,0.0,454,10,...,2.0,0.0,0,0,0,0,55,False,0,2016/2017
1,Aaron Lennon,None,None,0,0,6,0,0.3,142,3,...,1.0,0.0,1,0,0,0,60,True,0,2016/2017
2,Aaron Ramsey,None,None,0,0,5,0,4.9,16,8,...,3.0,23.0,2,0,0,0,80,True,0,2016/2017
3,Abdoulaye Doucouré,None,None,0,0,0,0,0.0,482,7,...,1.0,0.0,0,0,0,0,50,False,0,2016/2017
4,Adam Forshaw,None,None,0,0,3,0,1.3,286,6,...,1.0,0.0,1,0,0,0,45,True,1,2016/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109578,Tawanda Chirewa,None,None,0,0,0,0,0.0,750,149,...,1.0,0.0,0,92,109,17,45,True,0,2023/2024
109579,Tawanda Chirewa,None,None,0,0,0,0,0.0,750,160,...,1.0,0.0,0,44,94,50,45,True,0,2023/2024
109580,Justin Hubner,None,None,0,0,0,0,0.0,751,131,...,2.0,0.0,0,0,0,0,40,False,0,2023/2024
109581,Justin Hubner,None,None,0,0,0,0,0.0,751,149,...,1.0,0.0,0,344,375,31,40,True,0,2023/2024


In [29]:
full_data[full_data["name"] == "Erling Haaland"].head()

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
69738,Erling Haaland,None,None,0,3,48,1,4.8,318,10,...,0.0,73.0,13,0,0,0,115,False,0,2022/2023
70312,Erling Haaland,None,None,1,0,16,1,13.3,318,17,...,4.0,23.0,5,1113937,1143813,29876,116,True,0,2022/2023
70894,Erling Haaland,None,None,0,0,28,0,11.9,318,28,...,3.0,70.0,6,227317,368864,141547,117,False,0,2022/2023
71487,Erling Haaland,None,None,0,3,73,0,1.3,318,37,...,4.0,105.0,17,99625,231905,132280,117,True,0,2022/2023
72089,Erling Haaland,None,None,0,3,81,1,0.8,318,49,...,6.0,91.0,17,624916,675165,50249,118,True,0,2022/2023


In [118]:
full_data.loc[full_data['name'] == "Erling Haaland", 'position'] = "Forward"
haaland = full_data[full_data["name"] == "Erling Haaland"]
np.unique(haaland["element"])

array([318, 355])

In [35]:
full_data.loc[full_data["name"] == "Erling Haaland", "position"].iloc[0] 

'Forward'

In [194]:
driver_path = "/home/jasmine/Desktop/drivers/chromedriver-linux64/chromedriver"
# create a webdriver to automate searching players up
driver = webdriver.Chrome(executable_path=driver_path)
player_link = "https://www.premierleague.com/players"
# direct driver to this page
driver.get(player_link)
# click the accept all cookies button - one-time
cookie_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
cookie_button.click()
# find the search bar to search up players
search_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.ID, "search-input"))
)

time.sleep(5)

for player in unique_players[1479:]: 

    # clear any content already in the search bar
    search_input.clear()

    # search up the player
    search_input.send_keys(player)
    search_input.send_keys(Keys.RETURN)

    time.sleep(6)

    try:
        search_results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "player__name"))
        )
        first_result = search_results[0]
        first_result.click()

    except StaleElementReferenceException:
        # If the element is stale, re-locate it
        time.sleep(1)  # Add a small delay to ensure the page updates
        search_results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "player__name"))
        )
        first_result = search_results[0]
        first_result.click()

    except TimeoutException:
        print(f"No results found for {player}. Moving on to the next player.")
        driver.back()
        continue

    # get player position
    overview_info = driver.find_elements(By.CLASS_NAME, "player-overview__col")
    infos = [info.text for info in overview_info]
    pos_info = [i for i in infos if "Position" in i]
    position = pos_info[0].split("\n")[-1]
    # update position for this player in the main df
    full_data.loc[full_data['name'] == player, 'position'] = position

    # get club history for each season using BeautifulSoup
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, features = "html.parser")

    # getting rid of shortened names
    short_names = soup.find_all("span", class_="player-club-history__team-name--short")
    for short_name in short_names:
        short_name.decompose()

    # find table w club history
    table = soup.find("table")
    history_df = pd.read_html(str(table))[0]
    # table contains lots of addl. info, so drop it
    history_df = history_df.dropna(subset=["Season"])
    # we only want one club occurrence per season; some players have two if they transferred (e.g. Jesus)
    # we keep the first occurrence (most recent club of the two)
    history_df = history_df.drop_duplicates(subset=["Season"])

    # dataframe for the player (subset of full_data)
    player_df = full_data[full_data["name"] == player]
    # all seasons that this player has been in the PL
    player_seasons = np.unique(history_df["Season"])

    # for current player, fill in club season by season
    for season in player_seasons: 

        # get the club associated w given season (will be same for most players)
        club = history_df.loc[history_df['Season'] == season, 'Club'].iloc[0]
        full_data.loc[(full_data['name'] == player) & (full_data['season'] == season), 'club'] = club

    print((player, position, club))

    driver.back()

('Wesley Moraes', 'Forward', 'Aston Villa')
('Wesley Moraes Ferreira da Silva', 'Forward', 'Aston Villa')
('Weston McKennie', 'Midfielder', 'Leeds United (Loan)')
('Wilfred Ndidi', 'Midfielder', 'Leicester City')
('Wilfried Gnonto', 'Forward', 'Leeds United')
('Wilfried Zaha', 'Forward', 'Crystal Palace')
('Will Dennis', 'Goalkeeper', 'Bournemouth')
('Will Ferry', 'Midfielder', 'Southampton')
('Will Hughes', 'Midfielder', 'Crystal Palace')
('Will Norris', 'Goalkeeper', 'Burnley')
('Will Smallbone', 'Midfielder', 'Southampton')
('William Fish', 'Defender', 'Manchester United')
('William Osula', 'Forward', 'Sheffield United')
('William Saliba', 'Defender', 'Arsenal')
('William Smallbone', 'Defender', 'Arsenal')
('William Troost-Ekong', 'Defender', 'Watford')
('Willian Borges Da Silva', 'Forward', 'Fulham')
('Willian Borges da Silva', 'Forward', 'Fulham')
('Willian José Da Silva', 'Forward', 'Wolverhampton Wanderers (Loan)')
('Willy Boly', 'Defender', 'Nottingham Forest')
('Willy Caballer

In [151]:
# fix Ademipo Odubeko
# remove second Traore
# for all clubs: if there are parentheses, remove them and everything inside
# alexandre moreno lopera is alex moreno
# andrew robertson is wrong
# antony matheus dos santos is wrong (should just be Antony)
# benjamin chrisine doesn't exist? delete
# there are literally 4 bernardo silvas...first is Bernardo Fernandes Da Silva Junior
# U21s...might not want to keep
# Gabriel dos Santos Magalhães is wrong - delete it, there is another
# delete mark o'mahony or fill in manually

'Billy Koumetio'

In [195]:
full_data.to_csv("full_data.csv", index = False)

In [196]:
full_data["position"] = full_data["position"].apply(str)

In [200]:
np.unique(full_data["position"])

array(['Defender', 'Forward', 'Goalkeeper', 'Midfielder', 'None'],
      dtype=object)

In [204]:
full_data['position'] = full_data['position'].replace('None', np.nan)
full_data = full_data.dropna(subset=['position'])
full_data[full_data["position"] == "None"]

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season


In [205]:
np.unique(full_data["position"])

array(['Defender', 'Forward', 'Goalkeeper', 'Midfielder'], dtype=object)

In [267]:
full_data["club"] = full_data["club"].apply(str)
np.unique(full_data["club"])

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Fulham', 'Huddersfield Town', 'Hull City',
       'Leeds United', 'Leicester City', 'Liverpool', 'Luton Town',
       'Manchester City', 'Manchester United', 'Middlesbrough',
       'Newcastle United', 'Newcastle United (Loan)', 'Norwich City',
       'Nottingham Forest', 'Sheffield United', 'Southampton',
       'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur',
       'Watford', 'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers', 'Wolverhampton Wanderers (Loan)', 'nan'],
      dtype=object)

In [268]:
# remove all rows where player played for U21 team
full_data = full_data[~full_data['club'].str.contains('U21', na=False)]
full_data = full_data[~full_data['club'].str.contains('U18', na=False)]

In [269]:
np.unique(full_data["club"])

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Fulham', 'Huddersfield Town', 'Hull City',
       'Leeds United', 'Leicester City', 'Liverpool', 'Luton Town',
       'Manchester City', 'Manchester United', 'Middlesbrough',
       'Newcastle United', 'Newcastle United (Loan)', 'Norwich City',
       'Nottingham Forest', 'Sheffield United', 'Southampton',
       'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur',
       'Watford', 'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers', 'Wolverhampton Wanderers (Loan)', 'nan'],
      dtype=object)

In [279]:
full_data.to_csv("full_data.csv", index = False)

In [270]:
full_data['club'] = full_data['club'].str.replace(r'\(Loan\)', '', regex=True).str.strip()

In [271]:
np.unique(full_data["club"])

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Fulham', 'Huddersfield Town', 'Hull City',
       'Leeds United', 'Leicester City', 'Liverpool', 'Luton Town',
       'Manchester City', 'Manchester United', 'Middlesbrough',
       'Newcastle United', 'Norwich City', 'Nottingham Forest',
       'Sheffield United', 'Southampton', 'Stoke City', 'Sunderland',
       'Swansea City', 'Tottenham Hotspur', 'Watford',
       'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers', 'nan'], dtype=object)

In [236]:
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
0,Aaron Cresswell,West Ham United,Defender,0,0,0,0,0.0,454,10,...,2.0,0.0,0,0,0,0,55,False,0,2016/2017
1,Aaron Lennon,Everton,Midfielder,0,0,6,0,0.3,142,3,...,1.0,0.0,1,0,0,0,60,True,0,2016/2017
2,Aaron Ramsey,None,Midfielder,0,0,5,0,4.9,16,8,...,3.0,23.0,2,0,0,0,80,True,0,2016/2017
3,Abdoulaye Doucouré,Watford,Midfielder,0,0,0,0,0.0,482,7,...,1.0,0.0,0,0,0,0,50,False,0,2016/2017
4,Adam Forshaw,Middlesbrough,Midfielder,0,0,3,0,1.3,286,6,...,1.0,0.0,1,0,0,0,45,True,1,2016/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109569,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,10,0,14.4,715,120,...,2.0,3.0,2,389,465,76,49,True,0,2023/2024
109570,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,1,0,22,0,28.7,715,124,...,3.0,8.0,5,221,439,218,49,False,0,2023/2024
109571,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,6,0,1.7,715,131,...,2.0,2.0,2,952,1148,196,49,False,0,2023/2024
109572,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,3,0,0.8,715,149,...,1.0,2.0,1,934,1311,377,49,True,0,2023/2024


In [244]:
# manually fixing the dataset 
missing_club = full_data[full_data["club"] == "None"]
np.unique(missing_club["name"])

array(['Aaron Ramsey', 'Adam Smith', 'Ademipo Odubeko',
       'Alexandre Moreno Lopera', 'Andrew Robertson', 'Andriy Yarmolenko',
       'Antony Matheus dos Santos', 'Benjamin Chilwell',
       'Benjamin Chrisene', 'Benjamin White', 'Callum Scanlon',
       'Carlos Henrique Casimiro', 'Carlos Ribeiro Dias',
       'Conrad Egan-Riley', 'Crysencio Summerville', 'Daniel Bentley',
       'Daniel Chesters', "Daniel N'Lundulu",
       'Danilo dos Santos de Oliveira', 'Danny Ward',
       'Diogo Teixeira da Silva', 'Emerson Leite de Souza Junior',
       'Emerson Palmieri dos Santos', 'Fabio Henrique Tavares',
       'Fabricio Agosto Ramírez', 'Facundo Buonanotte',
       'Faustino Anjorin', 'Fernando Luiz Rosa', 'Fernando Marçal',
       'Francisco Jorge Tomás Oliveira',
       'Frederico Rodrigues de Paula Santos', 'Fábio Ferreira Vieira',
       'Gabriel dos Santos Magalhães', 'James Bree', 'Joe Taylor',
       'Jonathan Castro Otto', 'Jordan Henderson', 'Jordan Hugill',
       'Jorge Lui

In [258]:
# players whose searched names are different from conventional names
# to fix in loop and also in full_data
# wrong: list of player names used in search who yielded incorrect results
wrong = ["Andrew Robertson", "Alexandre Moreno Lopera", "Andriy Yarmolenko", "Antony Matheus dos Santos", "Benjamin Chilwell", 
        "Carlos Henrique Casimiro", "Carlos Ribeiro Dias", "Conrad Egan-Riley", "Crysencio Summerville", "Daniel Bentley", 
        "Daniel Chesters", "Danilo dos Santos de Oliveira", "Emerson Leite de Souza Junior", "Fabio Henrique Tavares", 
        "Fabricio Agosto Ramírez", "Facundo Buonanotte", "Faustino Anjorin", "Fernando Luiz Rosa", "Fernando Marçal", 
        "Francisco Jorge Tomás Oliveira", "Frederico Rodrigues de Paula Santos", "Fábio Ferreira Vieira", 
        "Gabriel dos Santos Magalhães", "James Bree", "Jonathan Castro Otto", "Jordan Henderson", "Jordan Hugill", 
        "Jorge Luiz Frello Filho", "Joseph Gomez", "Joseph Willock", "Josh Sims", "Joshua Sargent", "José Ignacio Peleteiro Romallo", 
        "Joshua Wilson-Esbrand", "Joseph Johnson", "Callum Scanlon", "Benjamin Chrisene", "Juan Camilo Hernández Suárez", 
        "Kell Watts", "Konstantinos Tsimikas", "Kyle Bartley", "Léo Bonatini", "Mahmoud Ahmed Ibrahim Hassan", 
        "Marcus Oliveira Alencar", "Mathias Jorgensen", "Matthew Cash", "Matthew Clarke", "Matthew Longstaff", "Matthew Pollock", 
        "Max Kilman", "Mohamed Dräger", "Norberto Bercique Gomes Betuncal", "Pelenda Joshua Dasilva", "Raphael Dias Belloli", 
        "Robbie Brady", "Rodrigo Hernandez", "Rui Pedro dos Santos Patrício", "Shaqai Forde", "Solomon March", "Victor da Silva", 
        "Vitor Ferreira", "Xande Nascimento da Costa Silva", "Yegor Yarmoliuk"]
# fixable: corresponding names that will yield correct results
fixable = ["Andy Robertson", "Álex Moreno", "Andrii Yarmolenko", "Antony", "Ben Chilwell", "Casemiro", "Cafú", "CJ Egan-Riley", 
          "Crysencio Summerville", "Dan Bentley", "Dan Chesters", "Danilo", "Emerson Royal", "Fabinho", "Fabri", "Facundo Buonanotte", 
          "Tino Anjorin", "Fernandinho", "Marçal", "Chiquinho", "Fred", "Fábio Vieira", "Gabriel Magalhães", "James Bree", "Jonny Otto", 
          "Jordan Henderson", "Jordan Hugill", "Jorginho", "Joe Gomez", "Joe Willock", "Josh Sims", "Josh Sargent", "Jota", 
          "Josh Wilson-Esbrand", "Joe Johnson", "Calum Scanlon", "Ben Chrisene", "Cucho Hernández", "Kelland Watts", "Kostas Tsimikas", 
          "Kyle Bartley", "Léo Bonatini", "Trézéguet", "Marquinhos", "Zanka", "Matty Cash", "Matt Clarke", "Matty Longstaff", 
          "Mattie Pollock", "Maximilian Kilman", "Mohamed Dräger", "Beto", "Josh Dasilva", "Raphinha", "Robert Brady", "Rodri", 
          "Rui Patrício", "Shaq Forde", "Solly March", "Vitinho", "Vitinha", "Xande Silva", "Yehor Yarmoliuk"]
# to_delete: players who are already accounted for under a different name OR who just don't exist - delete from full_data
to_delete = ["Benjamin White", "Daniel N'Lundulu", "Joe Taylor", "João Pedro Cavaco Cancelo", "Aaron Ramsey", "Adam Smith", "Mason Burstow", 
         "Matthew Smith", "Thomas McGill"]

In [248]:
full_data = full_data[~full_data["name"].isin(to_delete)]

In [263]:
for wrong_player, fix_player in zip(wrong, fixable): 
    
    full_data.loc[full_data['name'] == wrong_player, 'name'] = fix_player
    
missing_club = full_data[full_data["club"].isna()]
np.unique(missing_club["name"])

array(['Ademipo Odubeko', 'Andrii Yarmolenko', 'Andy Robertson', 'Antony',
       'Ben Chilwell', 'Ben Chrisene', 'Beto', 'CJ Egan-Riley', 'Cafú',
       'Calum Scanlon', 'Casemiro', 'Chiquinho', 'Crysencio Summerville',
       'Cucho Hernández', 'Dan Bentley', 'Dan Chesters', 'Danilo',
       'Danny Ward', 'Diogo Teixeira da Silva',
       'Emerson Palmieri dos Santos', 'Emerson Royal', 'Fabinho', 'Fabri',
       'Facundo Buonanotte', 'Fernandinho', 'Fred', 'Fábio Vieira',
       'Gabriel Magalhães', 'James Bree', 'Joe Gomez', 'Joe Johnson',
       'Joe Willock', 'Jonny Otto', 'Jordan Henderson', 'Jordan Hugill',
       'Jorginho', 'Josh Dasilva', 'Josh Sargent', 'Josh Sims',
       'Josh Wilson-Esbrand', 'Jota', 'Kaine Hayden', 'Kelland Watts',
       'Kostas Tsimikas', 'Kyle Bartley', 'Léo Bonatini', 'Marquinhos',
       'Marçal', 'Matt Clarke', 'Mattie Pollock', 'Matty Cash',
       'Matty Longstaff', 'Maximilian Kilman', 'Mohamed Dräger',
       'Raphinha', 'Robert Brady', 'Rodri'

In [264]:
driver_path = "/home/jasmine/Desktop/drivers/chromedriver-linux64/chromedriver"
# create a webdriver to automate searching players up
driver = webdriver.Chrome(executable_path=driver_path)
player_link = "https://www.premierleague.com/players"
# direct driver to this page
driver.get(player_link)
# click the accept all cookies button - one-time
cookie_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
cookie_button.click()
# find the search bar to search up players
search_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.ID, "search-input"))
)

time.sleep(5)

for player in fixable: 

    # clear any content already in the search bar
    search_input.clear()

    # search up the player
    search_input.send_keys(player)
    search_input.send_keys(Keys.RETURN)

    time.sleep(6)

    try:
        search_results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "player__name"))
        )
        first_result = search_results[0]
        first_result.click()

    except StaleElementReferenceException:
        # If the element is stale, re-locate it
        time.sleep(1)  # Add a small delay to ensure the page updates
        search_results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "player__name"))
        )
        first_result = search_results[0]
        first_result.click()

    except TimeoutException:
        print(f"No results found for {player}. Moving on to the next player.")
        driver.back()
        continue

    # get player position
    overview_info = driver.find_elements(By.CLASS_NAME, "player-overview__col")
    infos = [info.text for info in overview_info]
    pos_info = [i for i in infos if "Position" in i]
    position = pos_info[0].split("\n")[-1]
    # update position for this player in the main df
    full_data.loc[full_data['name'] == player, 'position'] = position

    # get club history for each season using BeautifulSoup
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, features = "html.parser")

    # getting rid of shortened names
    short_names = soup.find_all("span", class_="player-club-history__team-name--short")
    for short_name in short_names:
        short_name.decompose()

    # find table w club history
    table = soup.find("table")
    history_df = pd.read_html(str(table))[0]
    # table contains lots of addl. info, so drop it
    history_df = history_df.dropna(subset=["Season"])
    # we only want one club occurrence per season; some players have two if they transferred (e.g. Jesus)
    # we keep the first occurrence (most recent club of the two)
    history_df = history_df.drop_duplicates(subset=["Season"])

    # dataframe for the player (subset of full_data)
    player_df = full_data[full_data["name"] == player]
    # all seasons that this player has been in the PL
    player_seasons = np.unique(history_df["Season"])

    # for current player, fill in club season by season
    for season in player_seasons: 

        # get the club associated w given season (will be same for most players)
        club = history_df.loc[history_df['Season'] == season, 'Club'].iloc[0]
        full_data.loc[(full_data['name'] == player) & (full_data['season'] == season), 'club'] = club

    print((player, position, club))

    driver.back()

('Andy Robertson', 'Defender', 'Liverpool')
('Álex Moreno', 'Defender', 'Aston Villa')
('Andrii Yarmolenko', 'Forward', 'West Ham United')
('Antony', 'Forward', 'Manchester United')
('Ben Chilwell', 'Defender', 'Chelsea')
('Casemiro', 'Midfielder', 'Manchester United')
('Cafú', 'Midfielder', 'Nottingham Forest')
('CJ Egan-Riley', 'Defender', 'Burnley U21')
('Crysencio Summerville', 'Forward', 'Leeds United')
('Dan Bentley', 'Goalkeeper', 'Wolverhampton Wanderers')
('Dan Chesters', 'Midfielder', 'West Ham United')
('Danilo', 'Midfielder', 'Nottingham Forest')
('Emerson Royal', 'Defender', 'Tottenham Hotspur')
('Fabinho', 'Midfielder', 'Liverpool')
('Fabri', 'Goalkeeper', 'Fulham')
('Facundo Buonanotte', 'Midfielder', 'Brighton & Hove Albion')
('Tino Anjorin', 'Midfielder', 'Chelsea')
('Fernandinho', 'Midfielder', 'Manchester City')
('Marçal', 'Defender', 'Wolverhampton Wanderers')
('Chiquinho', 'Forward', 'Wolverhampton Wanderers')
('Fred', 'Midfielder', 'Manchester United')
('Fábio Vie

In [276]:
nan_club = full_data[full_data["club"] == "nan"]
to_delete_again = np.unique(nan_club["name"])
to_delete_again

array(['Ademipo Odubeko', 'Danny Ward', 'Diogo Teixeira da Silva',
       'Emerson Palmieri dos Santos', 'James Bree', 'Jordan Hugill',
       'Josh Sims', 'Kaine Hayden', 'Kelland Watts', 'Léo Bonatini',
       'Matt Clarke', 'Mohamed Dräger'], dtype=object)

In [277]:
full_data = full_data[~full_data["name"].isin(to_delete_again)]

In [278]:
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
0,Aaron Cresswell,West Ham United,Defender,0,0,0,0,0.0,454,10,...,2.0,0.0,0,0,0,0,55,False,0,2016/2017
1,Aaron Lennon,Everton,Midfielder,0,0,6,0,0.3,142,3,...,1.0,0.0,1,0,0,0,60,True,0,2016/2017
2,Abdoulaye Doucouré,Watford,Midfielder,0,0,0,0,0.0,482,7,...,1.0,0.0,0,0,0,0,50,False,0,2016/2017
3,Adam Forshaw,Middlesbrough,Midfielder,0,0,3,0,1.3,286,6,...,1.0,0.0,1,0,0,0,45,True,1,2016/2017
4,Adam Lallana,Liverpool,Midfielder,1,2,33,0,33.7,205,8,...,3.0,57.0,11,0,0,0,70,False,1,2016/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107854,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,10,0,14.4,715,120,...,2.0,3.0,2,389,465,76,49,True,0,2023/2024
107855,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,1,0,22,0,28.7,715,124,...,3.0,8.0,5,221,439,218,49,False,0,2023/2024
107856,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,6,0,1.7,715,131,...,2.0,2.0,2,952,1148,196,49,False,0,2023/2024
107857,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,3,0,0.8,715,149,...,1.0,2.0,1,934,1311,377,49,True,0,2023/2024


In [9]:
full_data = pd.read_csv("full_data.csv")
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season,opponent_team_name,result
0,Aaron Cresswell,West Ham United,Defender,0,0,0,0,0.0,454,10,...,0,0,0,0,55,False,0,2016/2017,Chelsea,L
1,Aaron Lennon,Everton,Midfielder,0,0,6,0,0.3,142,3,...,1,0,0,0,60,True,0,2016/2017,Tottenham Hotspur,D
2,Abdoulaye Doucouré,Watford,Midfielder,0,0,0,0,0.0,482,7,...,0,0,0,0,50,False,0,2016/2017,Southampton,D
3,Adam Forshaw,Middlesbrough,Midfielder,0,0,3,0,1.3,286,6,...,1,0,0,0,45,True,1,2016/2017,Stoke City,D
4,Adam Lallana,Liverpool,Midfielder,1,2,33,0,33.7,205,8,...,11,0,0,0,70,False,1,2016/2017,Arsenal,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,10,0,14.4,715,120,...,2,389,465,76,49,True,0,2023/2024,Tottenham Hotspur,W
107085,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,1,0,22,0,28.7,715,124,...,5,221,439,218,49,False,0,2023/2024,Fulham,L
107086,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,6,0,1.7,715,131,...,2,952,1148,196,49,False,0,2023/2024,Arsenal,L
107087,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,3,0,0.8,715,149,...,1,934,1311,377,49,True,0,2023/2024,Burnley,W


In [5]:
full_data.dtypes

name                              object
club                              object
position                          object
assists                            int64
bonus                              int64
bps                                int64
clean_sheets                       int64
creativity                       float64
element                            int64
fixture                            int64
goals_conceded                     int64
goals_scored                       int64
ict_index                        float64
influence                        float64
kickoff_time         datetime64[ns, UTC]
minutes                            int64
opponent_team                      int64
own_goals                          int64
penalties_missed                   int64
penalties_saved                    int64
red_cards                          int64
round                              int64
saves                              int64
selected                           int64
team_a_score    

In [4]:
full_data["kickoff_time"] = pd.to_datetime(full_data["kickoff_time"])

### Current idea for this project
The aim is to create some kind of machine learning model who can take in your team's players, look at their stats from the directly previous game (and if possible, also a rolling average of multiple / all precious games), and give a etsimate of the points each player will score in the upcoming gameweek. 

#### Current challenges / questions / uncertainties: 
- I'd like to use the club that the player plays for as a predictor, but I'm unsure of how I would encode this, especially since clubs change each season and so the inputted club might be completely unseen before to the model. I would like for the model to simply disregard this feature if this occurs, but I'm not sure if this is possible.
- I would like for a way to put into consideration the upcoming opponent team's form / difficulty, but this isn't included in my dataset.
- What is bps?
- 

In [10]:
full_data.groupby('club')['total_points'].mean().sort_values(ascending=False)

club
Huddersfield Town          2.029126
Middlesbrough              1.923913
Manchester City            1.902338
Stoke City                 1.900000
Swansea City               1.841867
Liverpool                  1.745722
Hull City                  1.682635
Tottenham Hotspur          1.655354
Chelsea                    1.547296
Arsenal                    1.503270
Leicester City             1.488398
West Ham United            1.476945
Manchester United          1.473058
Sunderland                 1.443299
Crystal Palace             1.380864
Newcastle United           1.350427
Burnley                    1.321783
West Bromwich Albion       1.314639
Brighton & Hove Albion     1.308658
Aston Villa                1.304991
Southampton                1.280947
Bournemouth                1.222659
Everton                    1.195057
Brentford                  1.177663
Wolverhampton Wanderers    1.168249
Fulham                     1.089626
Leeds United               1.082783
Watford                

In [11]:
full_data[full_data["club"] == "Huddersfield Town"]

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season
8493,Aaron Mooy,Huddersfield Town,Midfielder,1,0,22,1,46.9,172,4,...,0.0,0.0,6,0,0,0,55,False,0,2017/2018
8623,Joe Lolley,Huddersfield Town,Forward,0,0,0,0,0.0,176,4,...,0.0,0.0,0,0,0,0,45,False,0,2017/2018
8626,Jonas Lössl,Huddersfield Town,Goalkeeper,0,2,31,1,0.0,163,4,...,0.0,0.0,9,0,0,0,45,False,0,2017/2018
8668,Zanka,Huddersfield Town,Defender,1,1,29,1,0.2,171,4,...,0.0,11.0,10,0,0,0,45,False,0,2017/2018
8698,Philip Billing,Huddersfield Town,Midfielder,0,0,14,1,2.9,175,4,...,0.0,7.0,3,0,0,0,45,False,0,2017/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19320,Aaron Mooy,Huddersfield Town,Midfielder,0,1,28,0,15.7,172,83,...,2.0,17.0,7,-55353,1871,57224,56,True,1,2017/2018
19458,Joe Lolley,Huddersfield Town,Forward,0,0,0,0,0.0,176,83,...,2.0,0.0,0,11,254,243,44,True,0,2017/2018
19461,Jonas Lössl,Huddersfield Town,Goalkeeper,1,0,26,0,10.0,163,83,...,2.0,0.0,5,-18120,2265,20385,46,True,0,2017/2018
19504,Zanka,Huddersfield Town,Defender,0,0,12,0,0.1,171,83,...,2.0,0.0,2,-29078,1046,30124,46,True,0,2017/2018


In [13]:
np.unique(full_data["opponent_team"])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [19]:
full_data[["name", "club", "kickoff_time", "season", "team_a_score", "team_h_score", "opponent_team"]]

,name,club,kickoff_time,season,team_a_score,team_h_score,opponent_team
0,Aaron Cresswell,West Ham United,2016-08-15 19:00:00+00:00,2016/2017,1.0,2.0,4
1,Aaron Lennon,Everton,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,17
2,Abdoulaye Doucouré,Watford,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,13
3,Adam Forshaw,Middlesbrough,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,14
4,Adam Lallana,Liverpool,2016-08-14 15:00:00+00:00,2016/2017,4.0,3.0,1
...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-11 12:30:00+00:00,2023/2024,1.0,2.0,18
107085,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-27 20:00:00+00:00,2023/2024,2.0,3.0,10
107086,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-02 15:00:00+00:00,2023/2024,1.0,2.0,1
107087,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-05 19:30:00+00:00,2023/2024,0.0,1.0,6


In [18]:
opps = pd.read_csv("opp_team_id_correspondance.csv")
opps

,season,team,team_name
0,2016-17,1,Arsenal
1,2016-17,2,Bournemouth
2,2016-17,3,Burnley
3,2016-17,4,Chelsea
4,2016-17,5,Crystal Palace
...,...,...,...
135,2022-23,16,Nott'm Forest
136,2022-23,17,Southampton
137,2022-23,18,Spurs
138,2022-23,19,West Ham


In [19]:
opps['season'] = opps['season'].str.replace('-', '/20')
opps

,season,team,team_name
0,2016/2017,1,Arsenal
1,2016/2017,2,Bournemouth
2,2016/2017,3,Burnley
3,2016/2017,4,Chelsea
4,2016/2017,5,Crystal Palace
...,...,...,...
135,2022/2023,16,Nott'm Forest
136,2022/2023,17,Southampton
137,2022/2023,18,Spurs
138,2022/2023,19,West Ham


In [20]:
temp = full_data[full_data["season"] == "2023/2024"]
temp2 = temp[["name", "club", "kickoff_time", "season", "team_a_score", "team_h_score", "opponent_team"]]
temp2

,name,club,kickoff_time,season,team_a_score,team_h_score,opponent_team
94027,Folarin Balogun,Arsenal,2023-08-12 12:00:00+00:00,2023/2024,1.0,2.0,16
94028,Folarin Balogun,Arsenal,2023-08-21 19:00:00+00:00,2023/2024,1.0,0.0,8
94029,Folarin Balogun,Arsenal,2023-08-26 14:00:00+00:00,2023/2024,2.0,2.0,10
94030,Folarin Balogun,Arsenal,2023-09-03 15:30:00+00:00,2023/2024,1.0,3.0,14
94031,Folarin Balogun,Arsenal,2023-09-17 15:30:00+00:00,2023/2024,1.0,0.0,9
...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-11 12:30:00+00:00,2023/2024,1.0,2.0,18
107085,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-27 20:00:00+00:00,2023/2024,2.0,3.0,10
107086,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-02 15:00:00+00:00,2023/2024,1.0,2.0,1
107087,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-05 19:30:00+00:00,2023/2024,0.0,1.0,6


In [21]:
new_teams_df = pd.read_csv("23-24_teams.csv")
new_teams_df = new_teams_df[["id", "name"]]
new_teams_df["season"] = "2023/2024"
new_teams_df = new_teams_df.rename(columns={"id": "team", "name": "team_name"})
new_teams_df

,team,team_name,season
0,1,Arsenal,2023/2024
1,2,Aston Villa,2023/2024
2,3,Bournemouth,2023/2024
3,4,Brentford,2023/2024
4,5,Brighton,2023/2024
5,6,Burnley,2023/2024
6,7,Chelsea,2023/2024
7,8,Crystal Palace,2023/2024
8,9,Everton,2023/2024
9,10,Fulham,2023/2024


In [22]:
opps = pd.concat([opps, new_teams_df])
opps

,season,team,team_name
0,2016/2017,1,Arsenal
1,2016/2017,2,Bournemouth
2,2016/2017,3,Burnley
3,2016/2017,4,Chelsea
4,2016/2017,5,Crystal Palace
...,...,...,...
15,2023/2024,16,Nott'm Forest
16,2023/2024,17,Sheffield Utd
17,2023/2024,18,Spurs
18,2023/2024,19,West Ham


In [23]:
unique_clubs = np.unique(full_data["club"])
diff = [team for team in np.unique(opps["team_name"]) if team not in unique_clubs]
unique_clubs, diff

(array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
        'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
        'Everton', 'Fulham', 'Huddersfield Town', 'Hull City',
        'Leeds United', 'Leicester City', 'Liverpool', 'Luton Town',
        'Manchester City', 'Manchester United', 'Middlesbrough',
        'Newcastle United', 'Norwich City', 'Nottingham Forest',
        'Sheffield United', 'Southampton', 'Stoke City', 'Sunderland',
        'Swansea City', 'Tottenham Hotspur', 'Watford',
        'West Bromwich Albion', 'West Ham United',
        'Wolverhampton Wanderers'], dtype=object),
 ['Brighton',
  'Cardiff',
  'Huddersfield',
  'Hull',
  'Leeds',
  'Leicester',
  'Luton',
  'Man City',
  'Man Utd',
  'Newcastle',
  'Norwich',
  "Nott'm Forest",
  'Sheffield Utd',
  'Spurs',
  'Stoke',
  'Swansea',
  'West Brom',
  'West Ham',
  'Wolves'])

In [24]:
diff.remove("Cardiff") # for some reason there are no Cardiff players?

In [25]:
corr = ["Brighton & Hove Albion", "Huddersfield Town", "Hull City", "Leeds United", "Leicester City", "Luton Town", 
        "Manchester City", "Manchester United", "Newcastle United", "Norwich City", "Nottingham Forest", "Sheffield United", 
        "Tottenham Hotspur", "Stoke City", "Swansea City", "West Bromwich Albion", "West Ham United", "Wolverhampton Wanderers"]
for club, full in zip(diff, corr): 

    opps.loc[(opps['team_name'] == club), 'team_name'] = full

np.unique(opps["team_name"])

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Cardiff', 'Chelsea',
       'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield Town',
       'Hull City', 'Leeds United', 'Leicester City', 'Liverpool',
       'Luton Town', 'Manchester City', 'Manchester United',
       'Middlesbrough', 'Newcastle United', 'Norwich City',
       'Nottingham Forest', 'Sheffield United', 'Southampton',
       'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur',
       'Watford', 'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [26]:
opps = opps.rename(columns={"team_name": "opponent_team"})
opps

,season,team,opponent_team
0,2016/2017,1,Arsenal
1,2016/2017,2,Bournemouth
2,2016/2017,3,Burnley
3,2016/2017,4,Chelsea
4,2016/2017,5,Crystal Palace
...,...,...,...
15,2023/2024,16,Nottingham Forest
16,2023/2024,17,Sheffield United
17,2023/2024,18,Tottenham Hotspur
18,2023/2024,19,West Ham United


In [27]:
opps.to_csv("opps_final.csv", index=False)

In [67]:
seasons = np.unique(opps["season"])
mapping_dict = dict()
for season in seasons: 

    season_df = opps[opps["season"] == season]
    mapping_dict[season] = dict(zip(season_df["team"], season_df["opponent_team"]))

In [68]:
for season, mapping in mapping_dict.items():
    
    full_data.loc[full_data['season'] == season, "opponent_team_name"] = full_data['opponent_team'].map(mapping)

In [69]:
full_data[["name", "club", "kickoff_time", "season", "team_a_score", "team_h_score", "opponent_team", "opponent_team_name"]]

,name,club,kickoff_time,season,team_a_score,team_h_score,opponent_team,opponent_team_name
0,Aaron Cresswell,West Ham United,2016-08-15 19:00:00+00:00,2016/2017,1.0,2.0,4,Chelsea
1,Aaron Lennon,Everton,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,17,Tottenham Hotspur
2,Abdoulaye Doucouré,Watford,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,13,Southampton
3,Adam Forshaw,Middlesbrough,2016-08-13 14:00:00+00:00,2016/2017,1.0,1.0,14,Stoke City
4,Adam Lallana,Liverpool,2016-08-14 15:00:00+00:00,2016/2017,4.0,3.0,1,Arsenal
...,...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-11 12:30:00+00:00,2023/2024,1.0,2.0,18,Tottenham Hotspur
107085,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-11-27 20:00:00+00:00,2023/2024,2.0,3.0,10,Fulham
107086,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-02 15:00:00+00:00,2023/2024,1.0,2.0,1,Arsenal
107087,Jean-Ricner Bellegarde,Wolverhampton Wanderers,2023-12-05 19:30:00+00:00,2023/2024,0.0,1.0,6,Burnley


In [75]:
full_data.to_csv("full_data.csv", index=False)

In [76]:
full_data["result"] = None
full_data

,name,club,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,season,opponent_team_name,result
0,Aaron Cresswell,West Ham United,Defender,0,0,0,0,0.0,454,10,...,0,0,0,0,55,False,0,2016/2017,Chelsea,None
1,Aaron Lennon,Everton,Midfielder,0,0,6,0,0.3,142,3,...,1,0,0,0,60,True,0,2016/2017,Tottenham Hotspur,None
2,Abdoulaye Doucouré,Watford,Midfielder,0,0,0,0,0.0,482,7,...,0,0,0,0,50,False,0,2016/2017,Southampton,None
3,Adam Forshaw,Middlesbrough,Midfielder,0,0,3,0,1.3,286,6,...,1,0,0,0,45,True,1,2016/2017,Stoke City,None
4,Adam Lallana,Liverpool,Midfielder,1,2,33,0,33.7,205,8,...,11,0,0,0,70,False,1,2016/2017,Arsenal,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,10,0,14.4,715,120,...,2,389,465,76,49,True,0,2023/2024,Tottenham Hotspur,None
107085,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,1,0,22,0,28.7,715,124,...,5,221,439,218,49,False,0,2023/2024,Fulham,None
107086,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,6,0,1.7,715,131,...,2,952,1148,196,49,False,0,2023/2024,Arsenal,None
107087,Jean-Ricner Bellegarde,Wolverhampton Wanderers,Midfielder,0,0,3,0,0.8,715,149,...,1,934,1311,377,49,True,0,2023/2024,Burnley,None


In [77]:
full_data.loc[(full_data["was_home"]) & (full_data["team_h_score"] > full_data["team_a_score"]), "result"] = "W"
full_data.loc[(full_data["was_home"]) & (full_data["team_h_score"] < full_data["team_a_score"]), "result"] = "L"
full_data.loc[(full_data["team_h_score"] == full_data["team_a_score"]), "result"] = "D"
full_data.loc[~full_data["was_home"] & (full_data["team_h_score"] < full_data["team_a_score"]), "result"] = "W"
full_data.loc[~full_data["was_home"] & (full_data["team_h_score"] > full_data["team_a_score"]), "result"] = "L"

In [81]:
full_data[["club", "was_home", "team_h_score", "team_a_score", "opponent_team_name", "kickoff_time", "result"]]

,club,was_home,team_h_score,team_a_score,opponent_team_name,kickoff_time,result
0,West Ham United,False,2.0,1.0,Chelsea,2016-08-15 19:00:00+00:00,L
1,Everton,True,1.0,1.0,Tottenham Hotspur,2016-08-13 14:00:00+00:00,D
2,Watford,False,1.0,1.0,Southampton,2016-08-13 14:00:00+00:00,D
3,Middlesbrough,True,1.0,1.0,Stoke City,2016-08-13 14:00:00+00:00,D
4,Liverpool,False,3.0,4.0,Arsenal,2016-08-14 15:00:00+00:00,W
...,...,...,...,...,...,...,...
107084,Wolverhampton Wanderers,True,2.0,1.0,Tottenham Hotspur,2023-11-11 12:30:00+00:00,W
107085,Wolverhampton Wanderers,False,3.0,2.0,Fulham,2023-11-27 20:00:00+00:00,L
107086,Wolverhampton Wanderers,False,2.0,1.0,Arsenal,2023-12-02 15:00:00+00:00,L
107087,Wolverhampton Wanderers,True,1.0,0.0,Burnley,2023-12-05 19:30:00+00:00,W


### Current idea for this project
The aim is to create some kind of machine learning model who can take in your team's players, look at their stats from the directly previous game (and if possible, also a rolling average of multiple / all precious games), and give a etsimate of the points each player will score in the upcoming gameweek.

### Current challenges / questions / uncertainties:
- I'd like to use the club that the player plays for as a predictor, but I'm unsure of how I would encode this, especially since clubs change each season and so the inputted club might be completely unseen before to the model. I would like for the model to simply disregard this feature if this occurs, but I'm not sure if this is possible.
- I would like for a way to put into consideration the upcoming opponent team's form / difficulty, but this isn't included in my dataset.
- What is bps?


In [82]:
full_data.columns

Index(['name', 'club', 'position', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'season', 'opponent_team_name',
       'result'],
      dtype='object')

In [92]:
# features / params I am unsure of - keeping just in case
to_keep = ["name", "element", "opponent_team", "season", "kickoff_time"]
# target variable
target = ["total_points"]
# specifically important features: position, club, ict_index, minutes, selected, result, transfers_balance, was_home, opponent_team_name
features = ["position", "assists", "club", "clean_sheets", "goals_scored", "ict_index", "minutes", "red_cards", "saves", 
            "selected", "result", "transfers_balance", "was_home", "opponent_team_name"]

In [93]:
final_data = full_data[to_keep + features + target]
final_data

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,ict_index,minutes,red_cards,saves,selected,result,transfers_balance,was_home,opponent_team_name,total_points
0,Aaron Cresswell,454,4,2016/2017,2016-08-15 19:00:00+00:00,Defender,0,West Ham United,0,0,0.0,0,0,0,14023,L,0,False,Chelsea,0
1,Aaron Lennon,142,17,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Everton,0,0,0.9,15,0,0,13918,D,0,True,Tottenham Hotspur,1
2,Abdoulaye Doucouré,482,13,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Watford,0,0,0.0,0,0,0,1051,D,0,False,Southampton,0
3,Adam Forshaw,286,14,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Middlesbrough,0,0,0.3,69,0,0,2723,D,0,True,Stoke City,1
4,Adam Lallana,205,1,2016/2017,2016-08-14 15:00:00+00:00,Midfielder,1,Liverpool,0,1,14.2,75,0,0,155525,W,0,False,Arsenal,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107084,Jean-Ricner Bellegarde,715,18,2023/2024,2023-11-11 12:30:00+00:00,Midfielder,0,Wolverhampton Wanderers,0,0,2.7,72,0,0,1231,W,389,True,Tottenham Hotspur,2
107085,Jean-Ricner Bellegarde,715,10,2023/2024,2023-11-27 20:00:00+00:00,Midfielder,1,Wolverhampton Wanderers,0,0,6.2,61,0,0,1516,L,221,False,Fulham,5
107086,Jean-Ricner Bellegarde,715,1,2023/2024,2023-12-02 15:00:00+00:00,Midfielder,0,Wolverhampton Wanderers,0,0,0.5,62,0,0,2527,L,952,False,Arsenal,2
107087,Jean-Ricner Bellegarde,715,6,2023/2024,2023-12-05 19:30:00+00:00,Midfielder,0,Wolverhampton Wanderers,0,0,0.4,11,0,0,3506,W,934,True,Burnley,1


In [90]:
np.unique(full_data["club"]) == np.unique(full_data["opponent_team_name"])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [94]:
final_data.to_csv("final_data.csv", index=False)